# Building a Spam Filter with Naive Bayes

In this project, we're going to build a spam filter for use on text messages using Naive Bayes. To classify messages as spam or non-spam, the Naive Bayes algorithm follows these steps:

- Learns how humans classify messages.
- Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
- Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).


## Dataset
To teach the algorithm how to classify messages, we'll use a dataset of 5,572 SMS messages that have been classified by humans. 

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). 

The data is unlabelled, so we'll name the two columns:
- ```Label```: whether the message is spam (```spam```) or not spam (```ham```).
- ```SMS```: the actual text message content

## Goal
Build a multinomial Naive Bayes algorithm to classify SMS messages as spam or non-spam. We want to achieve at least 80% accuracy.


In [1]:
import pandas as pd
import re


In [2]:
sms_messages = pd.read_csv('SMSSpamCollection', sep='\t',
                          header=None, names=['Label','SMS'])

In [3]:
sms_messages.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
sms_messages.shape

(5572, 2)

In [5]:
sms_messages['Label'].value_counts()

ham     4825
spam     747
Name: Label, dtype: int64

In [6]:
sms_messages['Label'].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

## Training and Testing Set

When loading the dataset, we see that about 86% of the SMS messages are not spam while the remaining 14% are spam. Let's split the data into a training set and a testing set so that at the end we can evaluate the success of our spam filter. We'll use 80% for training and 20% for testing. 

In [7]:
# randomize the entire dataset
random_sms = sms_messages.sample(frac=1,random_state=42)

# Calculate index for split
training_test_index = round(len(random_sms) * 0.8)

# split into training and testing
training = random_sms[:training_test_index].reset_index(drop=True)
testing = random_sms[training_test_index:].reset_index(drop=True)


In [8]:
training['Label'].value_counts(normalize=True)*100

ham     86.698071
spam    13.301929
Name: Label, dtype: float64

In [9]:
testing['Label'].value_counts(normalize=True)*100

ham     86.175943
spam    13.824057
Name: Label, dtype: float64

We see that we have approximately the same percentage of spam and non-spam messages in our training and our testing datasets as in the full dataset. This is exactly what we want.

## Letter Case and Punctuation

Let's begin the data cleaning process by removing the punctuation and bringing all the words to lower case. In the end, the algorithm will look calculate probabilities word-by-word, so we want to standardize the formatting.

In [10]:
training['SMS'] = training['SMS'].str.replace('\W', ' ')
training['SMS'] = training['SMS'].str.lower()

In [11]:
training['SMS'].head()

0    squeeeeeze   this is christmas hug   if u lik ...
1    and also i ve sorta blown him off a couple tim...
2    mmm thats better now i got a roast down me  i ...
3        mm have some kanji dont eat anything heavy ok
4    so there s a ring that comes with the guys cos...
Name: SMS, dtype: object

In [12]:
testing['SMS'] = testing['SMS'].str.replace('\W', ' ')
testing['SMS'] = testing['SMS'].str.lower()

In [13]:
testing['SMS'].head()

0    was playng 9 doors game and gt racing on phone...
1           i dont thnk its a wrong calling between us
2                          all e best 4 ur exam later 
3     hey what how about your project  started aha da 
4    dunno  my dad said he coming home 2 bring us o...
Name: SMS, dtype: object

## Creating the Vocabulary

In order to run the algorithm, we need to create a vocabulary of every word that is included in each message. We'll also want to know if words appear multiple times in the same message. To get this information in an easy to use way, let's add a column for every unique word in the messages. The row values will then reflect the number of times that word appears in that message. For example, if the word *computer* does not appear in a message, the value will be 0. But if the word *computer* appears twice in a different message, the value will be 2. 

To do this, let's first split each message into a list, then create a list of all unique words that appear. We'll see that we end up with 7816 unique words.

In [14]:
# transform each message from the SMS column into a list

training['SMS'] = training['SMS'].str.split()

vocabulary = []

for message in training['SMS']:
    for word in message:
        vocabulary.append(word)
        
vocabulary = set(vocabulary)

vocabulary = list(vocabulary)

In [15]:
print(len(vocabulary))

7816


## The Final Training Set

Now that we have our vocabulary, we need to use it to populate our dataset for analysis. To do this, we will first build a dictionary that we will then convert to a new DataFrame. The dictionary will include each unique word as keys, and the values will be a list of frequencies by message. 

In [16]:

# initialize the dictionary
word_counts_per_sms = {unique_word: [0] * len(training['SMS'])
                      for unique_word in vocabulary}

# loop over training['SMS'] using the enumerate function

for index, sms in enumerate(training['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [17]:
word_counts_df = pd.DataFrame(word_counts_per_sms)

analysis_df = pd.concat([training,word_counts_df],axis=1)

In [18]:
analysis_df.head()

,Label,SMS,0,00,000,008704050406,0121,01223585236,01223585334,0125698789,...,zoe,zogtorius,zoom,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[squeeeeeze, this, is, christmas, hug, if, u, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[and, also, i, ve, sorta, blown, him, off, a, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[mmm, thats, better, now, i, got, a, roast, do...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,"[mm, have, some, kanji, dont, eat, anything, h...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[so, there, s, a, ring, that, comes, with, the...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants

We now have our training data that we are ready to work with. As a reminder, we will use the following two equations to calculate the probabilities of the messages being spam or not-spam. 

$P(Spam|w_1, w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n} P(w_i | Spam)$

$P(Ham|w_1, w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n} P(w_i | Ham)$

And to calculate $P(w_i | Spam)$ and $P(w_i | Ham)$ we will use

$ P(w_i | Spam) = \frac{N_{w_i | Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{vocabulary}} $

$ P(w_i | Ham) = \frac{N_{w_i | Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{vocabulary}} $

Some of these values will be constant each message. Let's calculate these constants first:

- $P(Spam)$
- $P(Ham)$
- $N_{spam}$ - number of words in all spam messages
- $N_{ham}$ - number of words in all ham messages
- $N_{vocabulary}$

For now, we will let alpha = 1 for Laplace smoothing.

In [19]:
# Isolating spam and ham messages first
spam_messages = analysis_df[analysis_df['Label'] == 'spam']
ham_messages = analysis_df[analysis_df['Label'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(analysis_df)
p_ham = len(ham_messages) / len(analysis_df)

# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

## Calculating Parameters

Now, let's calculate $P(w_i|Spam)$ and $P(w_i|Ham)$ for each word. These values will be constant for the whole training set, so if we calculate these parameters now, we will save computational time in the future. 

In [20]:
p_spam_dict = {unique_word: 0
                      for unique_word in vocabulary}
p_ham_dict = {unique_word: 0
                      for unique_word in vocabulary}

In [21]:
# we isolated the spam and ham messages above as
# spam_messages and ham_messages

# calculate p_spam

for word in vocabulary:
    
    # calculate number of times the word appears
    # in spam and ham messages
    n_word_given_spam = spam_messages[word].sum()
    n_word_given_ham = ham_messages[word].sum()
    
    
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    
    p_spam_dict[word] = p_word_given_spam
    p_ham_dict[word] = p_word_given_ham

## Classifying a New Message

Now that we have calculated all the constants and parameters, we can create the spam filter. This filter will:

- take in as input a new message
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.


In [22]:

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in p_spam_dict:
            p_spam_given_message *= p_spam_dict[word]
            
        if word in p_ham_dict:
            p_ham_given_message *= p_ham_dict[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [23]:
classify('secret money prince code.')

P(Spam|message): 5.195728000170284e-16
P(Ham|message): 2.248098033852803e-17
Label: Spam


In [27]:
classify('Sounds good, Tom, then see u there.')

P(Spam|message): 4.74693224259436e-25
P(Ham|message): 2.7878169823581606e-21
Label: Ham


In [30]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')
         

P(Spam|message): 1.5223001843661562e-25
P(Ham|message): 1.2176099861344542e-27
Label: Spam


## Measuring the Spam Filter's Accuracy

Now that we have created our spam filter, let's test it on our testing set of 1,114 messages. We will modify the ```classify``` function we wrote last time. 

In [24]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in p_spam_dict:
            p_spam_given_message *= p_spam_dict[word]

        if word in p_ham_dict:
            p_ham_given_message *= p_spam_dict[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [25]:
testing['predicted'] = testing['SMS'].apply(classify_test_set)


In [26]:
correct = 0
total = len(testing)

for row in testing.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)


Correct: 956
Incorrect: 158
Accuracy: 0.8581687612208259
